### Словарь оценочных слов 
#### перевести его в dataframe для более удобной работы

In [2]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
import tqdm

In [3]:
m = Mystem()

In [4]:
df = pd.read_table('rusentilex.txt', sep=',',  encoding = 'utf-8', names=['Word', 'POS', 'InitialLem', 'Sentiment','Source', 'Ambiguity'])

In [5]:
df['POS'] = df['POS'].apply(lambda x: x.strip())
df['Sentiment'] = df['Sentiment'].apply(lambda x: x.strip())
df['Source'] = df['Source'].apply(lambda x: str(x).strip())

#### Нам нужны оценночные слова, так что фильтруем по opinion

In [6]:
df = df.loc[df['Source'] == 'opinion']

In [7]:
df.head()

,Word,POS,InitialLem,Sentiment,Source,Ambiguity
2,абракадабра,Noun,абракадабра,negative,opinion,NaN
3,абсурд,Noun,абсурд,negative,opinion,NaN
4,абсурдность,Noun,абсурдность,negative,opinion,NaN
5,абсурдный,Adj,абсурдный,negative,opinion,NaN
6,авантюра,Noun,авантюра,negative,opinion,NaN


#### Смотрим, что у нас в seed. Пригодится для последующего анализа

In [55]:
with open('seed.txt', 'r', encoding='utf-8') as f:
    text = f.readlines()
sentiment_list = [line.strip() for line in text]
sentiment_list

['аналогов нет',
 'подошва',
 'вежливый',
 'великолепный',
 'вкусный',
 'внимательный',
 'большой выбор',
 'вау',
 'неплохой',
 'идеальный',
 'неприятный',
 'понравилось',
 'рекомендуем',
 'на ура',
 'советуем',
 'отвратительный',
 'отличный',
 'положительный',
 'отрицательный',
 'приятный',
 'напоминать о себе',
 'спасибо',
 'радовать глаз',
 'на совесть',
 'молодцы',
 'придти еще раз',
 'орать',
 'улыбаться',
 'приветливый',
 'отличный',
 'замечательный']

### Тестовый корпус
#### извлекаем из xml id отзыва, сам отзыв, оценки за еду, интерьер и сервис. Создаём из извлечённых данных dataframe

In [120]:
with open('sent_train.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    #text = text.split('</review>')
    

In [121]:
import re
id_rew = re.findall('<review id="(.+?)">', text)
reviews = re.findall('<text>(.+?)</text>', text)
food =  re.findall('<food>(.+?)</food>', text)
interior = re.findall('<interior>(.+?)</interior>', text)
service = re.findall('<service>(.+?)</service>', text)

In [62]:
with open('reviews.txt', 'a', encoding='utf-8') as f:
    for r in reviews:
        f.write(r)

In [154]:
text_score = pd.DataFrame(np.column_stack([id_rew,reviews, food, interior, service]), 
                               columns=['id','review', 'food', 'interior', 'service'])

In [155]:
text_score.head()

,id,review,food,interior,service
0,17600,И пускай на меня не обижается наш прославленны...,8,8,8
1,23518,"- Здравствуйте. Виа Д’Арженто! - Добрый вечер,...",9,7,10
2,27221,"Советую вам уволить Вашего метродотеля Елену, ...",9,9,1
3,29097,отличный средне вековый интеръер. Приятное обс...,8,10,9
4,23065,Ужинали в ресторане Баден-Баден 6 марта . Импо...,10,8,8


#### Лемматизируем, переводим оценки в int

In [158]:
text_score['review'] = text_score['review'].apply(lambda x: m.lemmatize(x))

In [159]:
text_score['food'] = text_score['food'].apply(lambda x: int(x))
text_score['interior'] = text_score['interior'].apply(lambda x: int(x))
text_score['service'] = text_score['service'].apply(lambda x: int(x))

In [164]:
text_score['review'] = text_score['review'].apply(lambda x: ''.join(x))

#### Делим условно отзывы на позитивные и негативные по оценке за еду. Всё, что между 4 и 7, считаем нейтральными

In [163]:
pos = text_score.loc[text_score['food'] > 7]
neg = text_score.loc[text_score['food'] < 4]

#### Попытка рассчитать idf

In [ ]:
text = text_score['review'].tolist()

In [131]:
import math
def compute_idf(word, corpus):
        return math.log(len(corpus)/sum([1.0 for i in corpus if word in i]))

In [127]:
text = ''.join(text)

In [128]:
words = text.split()

In [133]:
dic = {}

In [ ]:
for i in words:
    dic[i] = compute_idf(i, words)

#### Сохраняем в csv нашу таблицу

In [165]:
text_score.to_csv('reviews_score.csv', sep=';',encoding='utf-8', header=True)